# Exploring Census Data

In [ ]:
from osgeo import ogr, osr
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
biz_att_df = pd.read_csv('./data/business_neighbors_atts_new.csv')

# Extract Counties and Zip Code

In [ ]:
geometry = [Point(xy) for xy in zip(biz_att_df.longitude, biz_att_df.latitude)]
crs = {'init': 'epsg:4326'} #http://www.spatialreference.org/ref/epsg/2263/
geo_df = gpd.GeoDataFrame(biz_att_df, crs=crs, geometry=geometry)

In [ ]:
# Get county based on lat lon
counties = gpd.read_file('data/counties/tl_2017_us_county.shp')
counties.crs = crs
counties.columns = [x.lower() for x in counties.columns]
# Rename column
counties.rename(columns={'countyfp':'fipscty'}, inplace=True)
# Get county of each.
geo_df = gpd.sjoin(geo_df, counties[['fipscty', 'geometry']], op='intersects', how='inner')
geo_df['fipscty'] = geo_df['fipscty'].apply(int)
geo_df.drop('index_right', axis=1, inplace=True)

In [ ]:
zipcodes = gpd.read_file('data/zipcodes/v104/zip_poly.gdb')
zipcodes = zipcodes.loc[zipcodes['STATE']=='NV']
zipcodes.columns = [x.lower() for x in zipcodes.columns]
zipcodes.rename(columns={'zip_code':'zipcode'}, inplace=True)
zipcodes.crs = crs
geo_df = gpd.sjoin(geo_df, zipcodes[['zipcode', 'geometry', 'pop_sqmi']], op='intersects',how='inner')
geo_df['zipcode'] = geo_df['zipcode'].apply(int)


# Load Census Data

In [ ]:
def add_county_data(df, year):
    cnty_df = pd.read_csv('data/census/cbp{}co.csv'.format(year))
    # Limit to restaurants only, naics codes:https://www.naics.com/six-digit-naics/?code=72
    cnty_df = cnty_df.loc[cnty_df['naics'] == '722///']
    # Drop unimportant columns
    cnty_keeplist = ['fipscty', 'emp', 'qp1', 'ap', 'est', 'n1_4', 'n5_9', 
                     'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n500_999', 'n1000']
    cnty_df = cnty_df[cnty_keeplist]
    cnty_df.columns = ['cn_y{}_{}'.format(year, x) for x in cnty_df.columns]
    cnty_df.rename(columns={'cn_y{}_fipscty'.format(year):'fipscty'}, inplace=True)
    return df.merge(cnty_df, on='fipscty')

def add_zip_data(df, year):
    # NOT ALL RESTAURANTS HAVE A MATCHING ZIP
    zip_df = pd.read_csv('data/census/zbp{}detail.csv'.format(year))
    # Limit to restaurants only, naics codes:https://www.naics.com/six-digit-naics/?code=72
    zip_df = zip_df.loc[zip_df['naics'] == '72----'] # Just doing all hospitality for now
    # Drop unimportant columns
    zip_keeplist = ['zip', 'est', 'n1_4', 'n5_9', 
                    'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n500_999', 'n1000']
    zip_df = zip_df[zip_keeplist]
    zip_df.columns = ['cn_y{}_zip_{}'.format(year, x) for x in zip_df.columns]
    zip_df.rename(columns={'cn_y{}_zip_zip'.format(year):'zipcode'}, inplace=True)
    return df.merge(zip_df, on='zipcode', how='left')



In [ ]:
year_list = [str(y).zfill(2) for y in range(4,17)]
for y in year_list:
    geo_df = add_county_data(geo_df, y)
    geo_df = add_zip_data(geo_df, y)

In [ ]:
# Missing zipcodes. There are 62 biz's in 89158 and there's not census data at all for that.
# Also 2 biz's in 89161, no census data.
# We'll have to interpolate or just forget it
zip_df = pd.read_csv('data/census/zbp05detail.csv')
geo_df['zipcode'].unique()[np.isin(geo_df['zipcode'].unique(), zip_df['zip'].unique(), invert=True)]

# Fill missing zip data

In [ ]:
# Using simple mean filling
census_cols = geo_df.columns.str.startswith('cn')
geo_df.loc[:, census_cols] = geo_df.loc[:, census_cols].fillna(geo_df.loc[:, census_cols].mean())

# Year-year diffs

In [ ]:
def add_year_diffs(df, year):
    """Add year-to-year diffs as percentage growth/decrease"""
    census_cols_suffixes = ['emp', 'qp1', 'ap', 'est', 'n1_4', 'n5_9',
       'n10_19', 'n20_49', 'n50_99', 'n100_249',
       'n500_999', 'n1000', 'zip_est', 'zip_n1_4',
       'zip_n5_9', 'zip_n10_19', 'zip_n20_49', 'zip_n50_99',
       'zip_n100_249', 'zip_n500_999', 'zip_n1000']
    year = str(year).zfill(2)
    prev_year = str(int(year) - 1).zfill(2)
    for col in census_cols_suffixes:
        new_col = 'cn_y{}_diff_{}'.format(year, col)
        prev_vals = df['cn_y{}_{}'.format(prev_year, col)].copy()
        prev_vals[prev_vals==0] = 1
        df[new_col] = 100*((df['cn_y{}_{}'.format(year, col)] - df['cn_y{}_{}'.format(prev_year, col)])/
                        prev_vals)
    
    return df

In [ ]:
for y in year_list[1:]:
    geo_df = add_year_diffs(geo_df, y)

# Extract year specific data

In [ ]:
geo_df_backup = geo_df.copy()

In [ ]:
geo_df = geo_df_backup.copy()

In [ ]:
success_df = pd.read_csv('./data/business_success_long.csv')
success_df['first_review'] = pd.to_datetime(success_df.first_review, format='%Y-%m-%d %H:%M:%S')
success_df.drop(['stars', 'is_open'], axis=1, inplace=True)

In [ ]:
# Merge and get open year
geo_df = geo_df.merge(success_df, on='business_id')
geo_df = geo_df.loc[geo_df['first_review'].dt.year <= 2015]

In [ ]:
# Generate random years for testing
# geo_df['open_year'] = np.random.randint(2005, 2016, geo_df.shape[0])

In [ ]:
def extract_open_year_data(df):
    census_cols_suffixes = ['emp', 'qp1', 'ap', 'est', 'n1_4', 'n5_9',
   'n10_19', 'n20_49', 'n50_99', 'n100_249',
   'n500_999', 'n1000', 'zip_est', 'zip_n1_4',
   'zip_n5_9', 'zip_n10_19', 'zip_n20_49', 'zip_n50_99',
   'zip_n100_249', 'zip_n500_999', 'zip_n1000']
    df['open_year_str'] = df['first_review'].dt.year.apply(str).str.replace('20', '')
    for col in census_cols_suffixes:
        df['cn_opyear_{}'.format(col)] = [r[1]['cn_y{}_{}'.format(r[1]['open_year_str'],col)] for r in df.iterrows()]
        df['cn_opyear_diff_{}'.format(col)] =[r[1]['cn_y{}_diff_{}'.format(r[1]['open_year_str'],col)] for r in df.iterrows()]
        
    return df

In [ ]:
final_df = extract_open_year_data(geo_df)

# Cutdown and save

In [ ]:
# Drop non-target year data
final_df.drop(final_df.columns[final_df.columns.str.startswith('cn_y')], inplace=True, axis=1)
final_df.columns
useful_cols = ['business_id'] +  \
    list(final_df.columns.values[final_df.columns.str.startswith('nn')]) +\
    list(final_df.columns.values[final_df.columns.str.startswith('cn')])
print(useful_cols)

In [ ]:
useful_cols = ['business_id', 'is_open', 'successful', 'stars', 'review_count', 'pop_sqmi', 'nn_count', 'nn_percent_rest', 'nn_all_percent_open',
               'nn_max_cat_sim', 'nn_min_cat_sim', 'nn_avg_stars', 'nn_max_stars',
               'nn_min_stars', 'nn_avg_review_count', 'nn_max_review_count', 
               'nn_min_review_count', 'nn_rest_avg_stars', 'nn_rest_max_stars', 
               'nn_rest_min_stars', 'nn_rest_avg_review_count', 'nn_rest_max_review_count', 
               'nn_rest_min_review_count', 'nn_weighted_avg_stars', 'nn_weighted_avg_review_count', 
               'nn_weighted_sum_stars', 'nn_weighted_sum_review_count', 'nn_rest_percent_open', 'nn_mean_cat_sim', 
               'cn_opyear_emp', 'cn_opyear_qp1', 'cn_opyear_ap', 'cn_opyear_est', 'cn_opyear_zip_est',
               'cn_opyear_diff_emp', 'cn_opyear_diff_qp1', 'cn_opyear_diff_ap', 'cn_opyear_diff_est', 'cn_opyear_diff_zip_est']
cutdown_df = final_df[useful_cols]
cutdown_df.shape

In [ ]:
# Fill missing NN data
nn_cols = cutdown_df.columns[cutdown_df.columns.str.startswith('nn_')]
cutdown_df.loc[:, nn_cols] = cutdown_df.loc[:, nn_cols].fillna(-1)

In [ ]:
cutdown_df.to_csv('./data/business_neighbors_census_atts.csv', index=False)

# Plotting

In [ ]:
def scatter_bfit(x, y, title, xlabel, ylabel, xlims=None, ylims=None):
    plt.figure(figsize=(8,8), facecolor='white')
    plt.scatter(x, y, alpha=0.25)
    plt.tick_params(axis='both', which='major', labelsize=12)
    m, b = np.polyfit(x, y, 1)
    if xlims != None:
        plt.xlim(xlims[0], xlims[1])
    else:
        xlims = (np.min(x), np.max(x))
    if ylims != None:
        plt.ylim(ylims[0], ylims[1])
    
    plt_x = np.arange(xlims[0], xlims[1], (xlims[1]-xlims[0])/len(x))
    plt.plot(plt_x, m*plt_x + b, '--', linewidth=2.5, color='orange')
    
    # Labels
    plt.title(title, size=20)
    plt.xlabel(xlabel, size=16)
    plt.ylabel(ylabel, size=16)
    plt.show()

In [ ]:
cutdown_df = pd.read_csv('./data/business_neighbors_census_atts.csv')
scatter_bfit(cutdown_df.cn_opyear_diff_zip_est, 
             cutdown_df.successful, 'foo',
            'bar', 'Rating',ylims=[0,1])